In [80]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('Scenario-A-merged_5s.csv')
df2 = pd.read_csv('Scenario-B-merged_5s.csv')
label = 'label'
# Concatenate the DataFrames 
full_df = pd.concat([df1, df2], ignore_index=True)

#change labels tor=1 other=0
full_df.loc[full_df[label] == 'TOR', 'label'] = 1
full_df.loc[full_df[label] != 1, 'label'] = 0
full_df.head(5)

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,10.0.2.15,53913,216.58.208.46,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,10.0.2.15,53913,216.58.208.46,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,10.0.2.15,53913,216.58.208.46,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
#Reformatt IP's by removing all decimals accept for the 1st one
def format_ip(ip_address):
    parts = ip_address.split('.')
    formatted_ip = '.'.join([parts[0], ''.join(parts[1:])])
    return float(formatted_ip)

full_df['Source IP'] = full_df['Source IP'].apply(format_ip)
full_df[' Destination IP'] = full_df[' Destination IP'].apply(format_ip)
full_df.head(5)

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0215,53913,216.582085,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,10.0215,53913,216.582085,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,10.0215,53913,216.582085,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,10.0215,53913,216.582085,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,10.0215,53913,216.582085,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
features = full_df.columns.tolist()
features.remove('label')

In [83]:
from scipy.stats import pointbiserialr
from math import sqrt

def getMerit(subset, label):
    k = len(subset)

    # average feature-class correlation
    rcf_all = []
    for feature in subset:
        coeff = pointbiserialr( df[label], df[feature] )
        rcf_all.append( abs( coeff.correlation ) )
    rcf = np.mean( rcf_all )

    # average feature-feature correlation
    corr = df[subset].corr()
    corr.values[np.tril_indices_from(corr.values)] = np.nan
    corr = abs(corr)
    rff = corr.unstack().mean()

    return (k * rcf) / sqrt(k + k * (k-1) * rff)

In [84]:
subset = ['Source IP', ' Source Port', ' Destination IP', ' Destination Port']
corr = full_df[subset].corr()
corr

,Source IP,Source Port,Destination IP,Destination Port
Source IP,1.000000,-0.457249,-0.351599,0.437683
Source Port,-0.457249,1.000000,0.519620,-0.468211
Destination IP,-0.351599,0.519620,1.000000,-0.628094
Destination Port,0.437683,-0.468211,-0.628094,1.000000


In [85]:
corr.values[np.tril_indices_from(corr.values)] = np.nan
corr

,Source IP,Source Port,Destination IP,Destination Port
Source IP,NaN,-0.457249,-0.351599,0.437683
Source Port,NaN,NaN,0.519620,-0.468211
Destination IP,NaN,NaN,NaN,-0.628094
Destination Port,NaN,NaN,NaN,NaN


In [86]:
#Get rid of all rows with infs or nans
full_df = full_df.replace([np.inf, -np.inf], np.nan).dropna()

#get rid of negatives
for column in full_df.columns:
    full_df[column] = full_df[column].apply(lambda x: max(0, x) if pd.notnull(x) else x)
    
#Ensure all values are of type float
full_df = full_df.astype(float) 


In [88]:
best_value = -1
best_feature = ''
for feature in features:
    coeff = pointbiserialr( full_df[label], full_df[feature] )
    abs_coeff = abs( coeff.correlation )
    if abs_coeff > best_value:
        best_value = abs_coeff
        best_feature = feature

print("Best first feature is %s with merit %.4f"%(best_feature, best_value))

Best first feature is  Protocol with merit 0.3700
